In [1]:
import warnings
warnings.filterwarnings('ignore',category=RuntimeWarning)
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import glob,os
from tqdm.auto import tqdm
import datetime
import proplot as plot
from netCDF4 import Dataset
from wrf import getvar
import os,sys,gc
sys.path.insert(1, '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/')
from tools import derive_var,read_and_proc,preproc_noensemble
from tools.mlr import mlr
from tools.preprocess import do_eof,preproc_maria

%matplotlib inline

plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',
                'tick.labelweight':'ultralight','gridminor.linestyle':'--','title.weight':'normal','linewidth':0.5})

/tmp/ipykernel_2951064/3383560018.py:22: ProplotWarning: rc setting 'linewidth' was renamed to 'meta.width' in version 0.8.
  plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',


In [2]:
!ls /work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/data/haiyan

extract.sh  mem10  mem12  mem14  mem16	mem18  mem2   mem3  mem5  mem7	mem9
mem1	    mem11  mem13  mem15  mem17	mem19  mem20  mem4  mem6  mem8	U.nc


#### Read in stuff

In [3]:
origin_path = '/scratch/06040/tg853394/tc/output/redux/maria/'
#############################################################
path_coor = '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/data/haiyan/U.nc'
coor = xr.open_dataset(path_coor)
#Function####################################################
import json,pickle
def read_some_azimuth_fields(fileloc=None,fieldname=None):
    dict_name = {}
    for inx,obj in enumerate(fileloc):
        field_read = xr.open_dataset(obj)
        dict_name[fieldname[inx]] = field_read
    return dict_name
def nearest_index(array, value):
    idx = (np.abs(array-value)).argmin()
    return idx.values
def save_to_pickle(loc=None,var=None):
    with open(loc,"wb") as f:
        pickle.dump(var,f)
    return None

#### Potential Temperature

In [4]:
def do_theta(originpath=None,coor_pres=coor.pres,savepath=None,number=None):
    """
    Derive theta and save to /scratch
    """
    ctrlvar_dict = read_some_azimuth_fields(fileloc=[originpath+'mem'+str(int(number))+'/azim_T_0-168hr_track1.nc'],fieldname=['T'])        
    lwcrf_theta = derive_var.do_theta(ctrlvar_dict['T']['T'],coor.pres)
    read_and_proc.save_to_pickle(savepath+'mem'+str(int(number))+'_theta',lwcrf_theta)
    del ctrlvar_dict,lwcrf_theta
    gc.collect()
    return None

#azim_T_0-168hr_track1.nc

In [5]:
a = read_some_azimuth_fields(fileloc=['/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/data/haiyan/'+'mem'+str(int(3))+'/azim_T_0-168hr_track1.nc'],fieldname=['T'])['T']

In [8]:
np.abs(a.radius-625).argmin()

<xarray.DataArray 'radius' ()>
array(208)

In [5]:
for i in tqdm(range(1,21)):
    do_theta(originpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/data/haiyan/',coor_pres=coor.pres,savepath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/testML/output/haiyan/',number=i)

  0%|          | 0/20 [00:00<?, ?it/s]

#### Radial/Tangential Velocities

In [9]:
def do_uradvtan(originpath=None,coor_pres=coor.pres,savepath=None,number=None,radius=None):
    """
    Derive urad/vtan and save to /scratch
    """
    ctrlvar_dict = read_some_azimuth_fields(fileloc=[originpath+'mem'+str(int(number))+'/azim_U_0-168hr_track1.nc',\
                                                     originpath+'mem'+str(int(number))+'/azim_V_0-168hr_track1.nc'],fieldname=['U','V'])
    r500 = nearest_index(ctrlvar_dict['U']['U'].radius,radius)
    ctrlhUrad,ctrlhVtan,_ = derive_var.ruppert_vtmax_calc(ctrlvar_dict['U']['U'].data,ctrlvar_dict['V']['V'].data,r500,ctrlvar_dict['U']['U'][:,0,:,:].azmiuth.values,None)
    read_and_proc.save_to_pickle(savepath+'mem'+str(int(number))+'_urad',ctrlhUrad)
    read_and_proc.save_to_pickle(savepath+'mem'+str(int(number))+'_vtan',ctrlhVtan)
    del ctrlvar_dict,ctrlhUrad,ctrlhVtan
    gc.collect()
    return None    

In [10]:
from tqdm import tqdm
for i in tqdm(range(1,21)):
    do_uradvtan(originpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/data/haiyan/',coor_pres=coor.pres,savepath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/testML/output/haiyan/',number=i,
               radius=625)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:50<00:00,  2.54s/it]
